# Q1

In [1]:
import pandas as pd

df = pd.read_csv('q1/click_event_log.csv')
df.head()

,user_id,ad_id,clicked_on_ad
0,20,308,0
1,35,270,0
2,36,168,0
3,21,479,0
4,86,321,0


In [3]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


ohe_user = OneHotEncoder()
ohe_ad = OneHotEncoder()
user_features = ohe_user.fit_transform(df['user_id'].reshape((-1, 1)))
ad_features = ohe_ad.fit_transform(df['ad_id'].reshape((-1, 1)))
X = hstack([user_features, ad_features]).todense()

y = df['clicked_on_ad'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [5]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, oob_score=True)
clf.fit(X_train, y_train)
print("OOB score: {}".format(clf.oob_score_))

predictions = clf.predict(X_test)
print(confusion_matrix(y_test, predictions))

OOB score: 0.83175
[[844  10]
 [146   0]]


a. Given a user_id and an ad_id, create a model that will predict the probability that the user will click on the ad. List any assumptions you make about the data when you create this model. What does this model output for user_id 12 and ad_id 10? What about user_id 50 and ad_id 177? Finally, what about user_id 13 and ad_id 241?

In [30]:
import numpy as np

users = np.array([12, 50, 13]).reshape((-1, 1)) - 1
ads = np.array([10, 177, 241]).reshape((-1, 1)) - 1

user_features = ohe_user.transform(users)
ad_features = ohe_ad.transform(ads)
features = hstack([user_features, ad_features]).todense()

clf.predict(features)

array([0, 0, 0], dtype=int64)

# Q2

In [1]:
import pandas as pd

df = pd.read_csv('q2/satelite_20.csv')
df.head()

,log_azimuth,log_altitiude,lens_size,months_in_operation,observed_image_output
0,2.351222,2.996789,8.041122,1,0
1,2.379440,3.050773,5.448220,3,0
2,2.441207,3.071607,2.134721,3,0
3,2.454324,3.127072,7.271947,1,0
4,2.465330,3.162127,6.854447,0,0


In [2]:
from collections import Counter

Counter(df['observed_image_output'])

Counter({0: 51, 1: 51, 2: 51})

In [3]:
from sklearn.model_selection import train_test_split

X = df.drop('observed_image_output', axis=1).values
y = df['observed_image_output'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix



clf = RandomForestClassifier(n_estimators=100, oob_score=True)
clf.fit(X, y)
print("OOB score: {}".format(clf.oob_score_))

predictions = clf.predict(X_test)
print(confusion_matrix(y_test, predictions)) 

OOB score: 0.758169934641
[[11  0  0]
 [ 0 12  0]
 [ 0  0  8]]


In [5]:
"""
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold, GridSearchCV


clf = RandomForestClassifier(n_estimators=100, oob_score=True)

clf.fit(X, y) print("OOB score: {}".format(clf.oobscore))
predictions = clf.predict(X_test) print(confusion_matrix(y_test, predictions))
""";

In [5]:
%%time
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold, GridSearchCV

n_estimators=range(50, 75, 100)
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(RandomForestClassifier(oob_score=True), param_grid, n_jobs=-1, cv=kfold)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 100 µs


In [13]:
grid_search.fit(X_train, y_train)
print("OOB score: {}".format(grid_search.best_estimator_.oob_score_))

OOB score: 0.631147540984


In [19]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import numpy as np

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = XGBClassifier(learning_rate=0.1)
subsample = np.linspace(0.7, 1.0, 3)
colsample_bylevel = np.linspace(0.6, 1.0, 3)
n_estimators=range(50, 200, 50)
max_depth = [1, 3, 5]
param_grid = dict(subsample=subsample, colsample_bylevel=colsample_bylevel, 
                  n_estimators=n_estimators, max_depth=max_depth)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(xgb_model, param_grid, n_jobs=-1, cv=kfold)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [50, 100, 150], 'subsample': array([ 0.7 ,  0.85,  1.  ]), 'max_depth': [1, 3, 5], 'colsample_bylevel': array([ 0.6,  0.8,  1. ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [21]:
df_test = pd.read_csv('q2/satelite_20_test.csv')
df_test.head()

,log_azimuth,log_altitiude,lens_size,months_in_operation
0,0.916291,1.609438,7.444006,1
1,2.014903,2.708050,4.910400,1
2,3.688879,4.382027,3.744001,3
3,3.806662,4.499810,8.171006,2
4,2.251292,2.302585,6.487642,3


In [22]:
grid_search.predict(df_test.values)

array([0, 0, 1, 2, 0, 0, 2, 2, 1, 1, 2, 2])

# Q3

In [1]:
import pandas as pd

df_dams = pd.read_csv('q3/dams.csv')
df_dams.head()

,from_dam,to_dam,amount,week
0,2,7,685,1
1,3,8,423,2
2,6,9,1026,3
3,7,9,817,4
4,3,8,2858,5


In [2]:
from collections import defaultdict

dam_connections = defaultdict(list)

for from_dam, to_dam, amount in df_dams.drop('week', axis=1).values:
    connection_pair = tuple(sorted([from_dam, to_dam]))
    dam_connections[connection_pair].append(amount)

# What do you mean by schematic

In [4]:
dam_volume = defaultdict(dict)
all_dams = set(df_dams.from_dam)

for dam in all_dams:
    dam_volume[dam][0] = 10000

for from_dam, to_dam, amount, week in df_dams.values:
    if from_dam != to_dam:
        dam_volume[from_dam][week] = dam_volume[from_dam][week - 1] - amount
        dam_volume[to_dam][week] = dam_volume[to_dam][week - 1] + amount
    else:
        dam_volume[from_dam][week] = dam_volume[from_dam][week - 1]
    
    for dam in (all_dams - set([from_dam, to_dam])):
        dam_volume[dam][week] = dam_volume[dam][week - 1]
        
    assert sum(dam_volume[dam][week] for dam in all_dams) == 100000, 'problem' # make sure all water is conserved

In [8]:
df_log = pd.read_csv('q3/foremen_log.csv')

def get_cracked_dams(string):
    if string == 'none':
        return None
    else:
        return list(int(number) for number in string.split(', '))

df_log['dams_cracked'] = df_log['dams_cracked'].apply(get_cracked_dams)

pd.DataFrame(dam_volume).head() # ledger of dam volume

,1,2,3,4,5,6,7,8,9,10
0,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
1,10000,9315,10000,10000,10000,10000,10685,10000,10000,10000
2,10000,9315,9577,10000,10000,10000,10685,10423,10000,10000
3,10000,9315,9577,10000,10000,8974,10685,10423,11026,10000
4,10000,9315,9577,10000,10000,8974,9868,10423,11843,10000


In [9]:
volume_in_cracked_dams = defaultdict(list)
uncracked_dam_volumes = dict()

for week, dams_cracked in df_log.values:
    if dams_cracked:
        for dam in dams_cracked:
            volume_in_cracked_dams[dam].append(dam_volume[dam][week])
        uncracked_dams = all_dams - set(dams_cracked)
        uncracked_dam_volumes[week] = sorted(dam_volume[dam][week] for dam in uncracked_dams)[-1]
    elif dams_cracked is None:
        uncracked_dam_volumes[week] = sorted(dam_volume[dam][week] for dam in all_dams)[-1]
    else:
        "Print unexpected error"

In [10]:
dam_threshold_no_crack = max(uncracked_dam_volumes.values())
dam_threshold_crack = min(reduce(lambda x, y: x + y, volume_in_cracked_dams.values()))

In [13]:
def predict_crack(dam_volume_list, dam_threshold_no_crack, dam_threshold_crack):
    cracked_dams = []
    undetermined_dams = []
    for dam_number, volume in enumerate(dam_volume_list, 1):
        if volume >= dam_threshold_crack:
            cracked_dams.append(dam_number)
        elif volume > dam_threshold_no_crack:
            undetermined_dams.append(dam_number)
    
    if (not cracked_dams and not undetermined_dams):
        return None
    elif not undetermined_dams:
        return cracked_dams
    else:
        return cracked_dams, undetermined_dams

dam_predictions = pd.DataFrame(dam_volume).apply(lambda row: predict_crack(row, dam_threshold_no_crack, dam_threshold_crack), axis=1)
print(pd.Series(dam_predictions)[1:31].values == df_log['dams_cracked'].values).all()

True


In [15]:
dam_predictions
# if None, then no dams are broken
# else, there can be 1 or 2 lists. The first list is known cracked dams. If the second list exists,
# it would be dams that are not known whether to be cracked or not since they are below the minimum
# cracked dam volume but also above the highest known uncracked dam volume

0                 None
1                 None
2                 None
3                 None
4                 None
5                 None
6                 None
7                 None
8                 None
9                 None
10                None
11                 [9]
12                 [9]
13                 [9]
14                 [9]
15                 [9]
16                 [9]
17                 [9]
18                 [9]
19                 [4]
20              [4, 8]
21           [4, 8, 9]
22           [4, 8, 9]
23           [4, 8, 9]
24           [4, 8, 9]
25              [4, 8]
26              [4, 8]
27              [4, 8]
28                 [8]
29                 [8]
            ...       
66    ([1, 4, 8], [6])
67       ([1, 8], [6])
68       ([1, 8], [6])
69       ([1, 8], [6])
70       ([1, 8], [6])
71       ([1, 8], [6])
72       ([1, 8], [6])
73       ([1, 8], [6])
74              [1, 8]
75              [1, 8]
76              [1, 8]
77              [1, 8]
78       ([

# Deprecated

In [4]:
from sklearn import naive_bayes

clf = naive_bayes.BernoulliNB()
clf.fit(X, y)

predictions = clf.predict(X)
confusion_matrix(y, clf.predict(X))

array([[4247,    3],
       [ 742,    8]])

In [6]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X, y)

predictions = clf.predict(X)
print(confusion_matrix(y, predictions))

[[4248    2]
 [ 745    5]]
